<a href="https://colab.research.google.com/github/aghosh92/SISSO_sGP/blob/main/physics_informed_featurization_QM9_1k_molecules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook shows how to generate ocmbination of nonlinear funtionalized features and rank them. The dataset used here contains the first 1,000 molecules as present in QM9 dataset.

This is the first step to generate meaningful hypotheses.

Notebook prepared by Ayana Ghosh

Email: research.aghosh@gmail.com

Install packages

In [1]:
!pip install -q matminer 
!pip install -q automatminer 

     |████████████████████████████████| 1.4 MB 4.7 MB/s 
     |████████████████████████████████| 66 kB 3.2 MB/s 
     |████████████████████████████████| 2.6 MB 36.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 98 kB 8.4 MB/s 
     |████████████████████████████████| 109 kB 39.6 MB/s 
     |████████████████████████████████| 515 kB 75.0 MB/s 
     |████████████████████████████████| 70 kB 8.3 MB/s 
     |████████████████████████████████| 561 kB 44.7 MB/s 
     |████████████████████████████████| 235 kB 71.5 MB/s 
     |████████████████████████████████| 555 kB 73.8 MB/s 
     |████████████████████████████████| 463 kB 5.3 MB/s 
     |████████████████████████████████| 265 kB 67.0 MB/s 
     |████████████████████████████████| 896 kB 36.2 MB/s 
     |████████████████████████████████| 7.0 MB 47.4 MB/s 
     |█████████████████████

Import essential packages

In [2]:
import os
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.linear_model import Lasso
from matminer.featurizers.function import FunctionFeaturizer
from automatminer import DataCleaner

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.unsupervised module is  deprecated in version 0.22 and w

In [3]:
#@title Helper functions for featurization
def get_data(selected_feature_list):
    
    function_featurizer = FunctionFeaturizer(multi_feature_depth=2,combo_function=np.sum)
    function_featurizer.set_n_jobs(4)
    function_featurizer=function_featurizer.fit(df_x[selected_feature_list])
    df_combined=function_featurizer.featurize_dataframe(df_x[selected_feature_list],selected_feature_list)

    df_combined[target] = df[target]
    df_combined=df_combined.replace([np.inf,-np.inf],np.nan)
    df_combined=df_combined.dropna(axis=1)
    df_combined=df_combined.drop(columns=selected_feature_list,axis=1)
    #save file 
    #df_combined.to_csv('filename.csv')

    P = df_combined[target].values
    df_combined = df_combined.loc[:, df_combined.columns != target]

    return P, df_combined

def lasso_fit(lam, P, D, feature_list):
    #LASSO
    #D_standardized = ss.zscore(D)
    lasso =  Lasso(alpha=lam)
    lasso.fit(D, P) 
    coef =  lasso.coef_
    
    # get strings of selected features
    selected_indices = coef.nonzero()[0]
    selected_features = [feature_list[i] for i in selected_indices]
    
    # get RMSE of LASSO model
    P_predict = lasso.predict(D)

    return coef,selected_features

Mount google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Read in dataset

In [5]:
!gdown https://drive.google.com/uc?id=18rx5GgyJoXE7os5hA3qDuFCH9BS3ZNWy
df = pd.read_csv('/content/featurized_1k.csv')

Downloading...
From: https://drive.google.com/uc?id=18rx5GgyJoXE7os5hA3qDuFCH9BS3ZNWy
To: /content/featurized_1k.csv
100% 90.0k/90.0k [00:00<00:00, 54.5MB/s]


In [ ]:
# #df_valid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/molecules_feat/featurized_3k_test.csv')
# !gdown https://drive.google.com/uc?id=1Ax4_-8gdjgKdzByNQF4kK2o0jrIUuWrU
# df_valid = pd.read_csv('/content/featurized_3k_test.csv')

Downloading...
From: https://drive.google.com/uc?id=1Ax4_-8gdjgKdzByNQF4kK2o0jrIUuWrU
To: /content/featurized_3k_test.csv
100% 253k/253k [00:00<00:00, 99.5MB/s]


In [6]:
feat_endp = ['smiles','tpsa', 'mol_wt', 'mole_logp', 'enthalpy']
#df = pd.concat([df1[feat_endp], df_valid[feat_endp]])

In [7]:
df.columns

Index(['Unnamed: 0', 'smiles', 'homo', 'lumo', 'gap', 'mol_wt', 'nhoh_ct',
       'valence', 'mole_logp', 'tpsa', 'enthalpy'],
      dtype='object')

In [8]:
target   = 'enthalpy'
selected_feature_list = ['tpsa', 'mol_wt', 'mole_logp']
df_x = df[selected_feature_list]

In [9]:
selected_feature_list = df_x.columns

In [10]:
selected_feature_list

Index(['tpsa', 'mol_wt', 'mole_logp'], dtype='object')

This step takes a while to run

In [11]:
print('generating functionalized data...')
P, df_D = get_data(selected_feature_list)
features_list = df_D.columns.to_list()
D = df_D.values

generating functionalized data...


FunctionFeaturizer: 100%|██████████| 1000/1000 [07:26<00:00,  2.24it/s]


In [12]:
alpha = 0.2

coef, selected_features = lasso_fit(alpha, P, D, features_list)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_coordinate_descent.py:474: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 52883.470413157636, tolerance: 85.48249451032828
  model = cd_fast.enet_coordinate_descent(


In [13]:
print("alpha: %.3f\t dimension of descriptor: %s" 
      %(alpha, len(selected_features)))

lasso_features=pd.DataFrame({'features':np.array(selected_features), 
                             'abs(nonzero_coefs_LASSO)': np.abs(coef[coef.nonzero()])}).sort_values(by='abs(nonzero_coefs_LASSO)',
                                                                                                    ascending=False)
print(lasso_features.head(n=10))
# save the combinations of features
#lasso_features.to_csv('filename.csv')

alpha: 0.200	 dimension of descriptor: 116
                        features  abs(nonzero_coefs_LASSO)
64       mole_logp + 1/log(tpsa)                  6.191368
68  exp(mole_logp) + 1/log(tpsa)                  2.027891
65    mole_logp**2 + 1/log(tpsa)                  1.103968
77        mole_logp + exp(-tpsa)                  0.811840
81   exp(mole_logp) + exp(-tpsa)                  0.297845
51     mole_logp**3 + sqrt(tpsa)                  0.244694
48           mole_logp**3 + tpsa                  0.105495
3                      mol_wt**2                  0.032640
46            tpsa + 1/mole_logp                  0.022819
0                        tpsa**2                  0.013248


We will use few of these combinations as analytical models after balancing dimensions to predict formation enthalpy (Hartree/g/mol) within Bayesian optimization.